In [49]:
import random
import nltk

In [50]:
BOT_CONFIG = {
    'intents':{
        'hello':{
            'examples':['Привет!','Хай!', 'Хеллоу', 'Утречко'],
            'responces':['Здравствуйте!', 'Доброго дня', 'Салют!', 'Прувэт!!']
        },
        'howareyou':{
            'examples':['Как дела ?','Как настроение?', 'Всё в порядке?'],
            'responces':['Отлично!', 'Хорошо', 'Грустно было вез Вас']
        },
        'weather':{
            'examples':['Какая погода?','Погода', 'Прогноз погоды'],
            'responces':['Солнечно', 'Дождь', 'Снег', 'Ветренно', 'Ясная']
        },
        'compliment':{
            'examples':['Спасибо!','Молодец!', 'Ты лучший'],
            'responces':['Всегда пожалуйста', 'Обращайтесь', 'Рад был помочь Вам!!']
        },
        'bye':{
            'examples':['Пока!','Увидимся!', 'Бай-бай!', 'Чао'],
            'responces':['До свиданья', 'Приходите ещё', 'Рад был познакомиться', 'До завтра!']
        }
    }
}

In [51]:
def clean_text(text):
  output_text = ''
  for char in text.lower():
    if char in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      output_text = output_text + char
  return output_text

def get_intent(text):
  for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
      text1 = clean_text(example)
      text2 = clean_text(text)
      if len(text1) != 0 and len(text2) != 0:
        if nltk.edit_distance(text1, text2)/ max(len(text1), len(text2)) < 0.4:
          return intent
  
  return 'Не удалось определить, что вы сказали'

In [52]:
def bot(input_text):
  intent = get_intent(input_text)
  if intent != 'Не удалось определить, что вы сказали':
    return random.choice(BOT_CONFIG['intents'][intent]['responces'])
  else:
    return 'Я ничего не понял! Повторите попытку.'


In [48]:
input_text = ''
while input_text != 'Завершите работу':
  input_text = input()
  print(bot(input_text))

превед
Доброго дня
Завершите работу
Я ничего не понял! Повторите попытку.
